# Исследование надёжности заёмщиков

Для анализа предоставлены данные кредитного отдела банка с информацией о семейном положении и количестве детей клиентов, а также статистикой их платежеспособности.

Цель исследования: определить факторы, которые влияют на невыполнение обязательств по погашению кредитной задолженности.

Согласно документации:

* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов (1 - да, 0 - нет);
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План работы

1. [Изучение](#1)

2. [Предобработка данных](#2) 
    
    2.1 [Обработка пропусков](#2.1)
    
    2.2 [Замена типа данных](#2.2)
    
    2.3 [Обработка дубликатов](#2.3)
    
    2.4 [Лемматизация](#2.4)
    
    2.5 [Категоризация данных](#2.5)
         
3. [Ответы на вопросы](#3) 

    3.1 [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#3.1)
    
    3.2 [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#3.2)
    
    3.3 [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#3.3)
    
    3.4 [Как разные цели кредита влияют на его возврат в срок?](#3.4)

4. [Общие выводы](#4)

<a name="1"><h2>1 Изучение данных</h2></a>

Импортируем библиотеки:

In [1]:
import pandas as pd
from pymystem3 import Mystem

Прочитаем данные и сохраним в переменную data:

In [2]:
data = pd.read_csv('/datasets/data.csv')

Выведем на экран первые 10 строк и просмотрим общую информацию о таблице:

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. В каждой строке таблицы данные, которые содержат информацию о клиенте, всего 21525 записей.

<div style="border: solid green 2px; padding: 20px">

**Выводы**
    
* Количество значений в столбцах 'days_employed' и 'total_income' различается, значит в данных есть пропуски.
* В столбце 'days_employed' отрицательные значения, вероятнеее всего такая ошибка связана с человеческим фактором, так как трудовой стаж не может быть отрицательным, поэтому нужно заменить значения на абсолютные.
* В столбце 'education' символы записаны по-разному, следует привести их к общему виду.
* Для столбцов с уровнем образования и семейным положением есть колонки с цифровыми обозначениями, поэтому к этим данным можно будет создать словари.</div>

<a name="2"><h2>2 Предобработка данных</h2></a>

<a name="2.1"><h3>2.1 Обработка пропусков</h3></a>

Подсчитаем количество пропусков:

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах примерно 10% пропусков, удаление такого объема значений может привести к существенным искажениям показателей, поэтому их лучше заменить на среднее значение или медиану.

Сначала поменяем отрицательные значения в столбце 'days_employed' на абсолютные:

In [6]:
data['days_employed'] = data['days_employed'].apply(abs)

Определим, каким значением лучше заменять, подсчитаем медиану и среднее значение:

In [7]:
round(data['days_employed'].median(),2)

2194.22

In [8]:
round(data['days_employed'].mean(),2)

66914.73

Значения различаются значительно. Чтобы данные были более корректные, лучше сгруппировать клиентов по возрасту, так как от этого зависит стаж работы, и затем взять значение медианы.

Циклом рассчитываем медиану для каждого возраста и заменяем строки с пропусками в столбце 'days_employed':

In [9]:
for age in data['dob_years'].unique():
  median = data.loc[data['dob_years'] == age, 'days_employed'].median()
  data.loc[(data['days_employed'].isna()) & (data['dob_years'] == age), 'days_employed'] = median

Проверяем, что пропусков в колонке с количеством дней стажа больше нет:

In [10]:
data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбце 'days_employed' пропусков не осталось, теперь обработаем пропуски в столбце с ежемесячным доходом 'total_income'. Выведем первые строки таблицы, чтобы принять решение, какими данными лучше заполнить пустые строки в этом столбце.

In [11]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Ежемесячный доход может зависить от образования и статуса сотрудника, поэтому сгруппируем клиентов по этим критериям и тоже заполним значением медианы. Но так как в столбце 'education' видно, что используется разный регистр, то сначала переведём все символы в нижний регистр.

In [12]:
data['education'] = data['education'].str.lower()

Проверяем результат:

In [13]:
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Проверим уникальные значения в столбце 'income_type':

In [14]:
data['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)


В колонке c уровнем образования дубликатов нет.

Создаём DataFrame с группировкой по типу занятости и по уровню образования клиентов:

In [15]:
group_data = (data.groupby(['income_type','education'])
.agg({'total_income': 'median'}).rename(columns = {'total_income': 'median_total_income'}))

Объединяем таблицы data и group_data:

In [16]:
data = data.merge(group_data, on = ['income_type','education'])

Заменяем медианным значением ежемесячного дохода строки с пропусками в столбце 'income_type':

In [17]:
data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']

Проверяем, что значений с пропусками больше нет:

In [18]:
data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
median_total_income    0
dtype: int64

В таблице отсутствуют пропущенные значения.

Теперь столбец 'median_total_income' не нужен и его можно удалить:

In [19]:
data = data.drop('median_total_income', axis=1)

Проверим, что столбец был удален:

In [20]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
2,0,2369.999720,33,высшее,0,гражданский брак,1,M,сотрудник,0,90410.586745,строительство недвижимости
3,0,272.981385,21,высшее,0,гражданский брак,1,M,сотрудник,0,128265.720871,сыграть свадьбу
4,0,529.191635,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848.983691,строительство собственной недвижимости


<div style="border: solid green 2px; padding: 20px">
    
**Вывод**

Все имевшиеся пропуски в столбцах 'days_employed' и 'total_income' были обработаны и заменены на медианные значения по выбранным критериям.</div>

<a name="2.2"><h3>2.2 Замена типа данных</h3></a>

В столбцах 'days_employed' и 'total_income' заменим тип 'float' на 'int' методом astype, так как такой вид данных более читабелен:

In [21]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверим результат:

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


<div style="border: solid green 2px; padding: 20px">

**Вывод**
    
Тип значений в столбцах с общим трудовым стажем и ежемесячным доходом изменён на целочисленный.</div>

<a name="2.3"><h3>2.3 Обработка дубликатов</h3></a>

Посчитаем явные дубликаты методами duplicated() и sum():

In [23]:
data.duplicated().sum()

71

В таблице дублируется 71 запись, удалим дубликаты и перезапишем индексы:

In [24]:
data = data.drop_duplicates().reset_index(drop=True)

Проверяем, что дубликаты были удалены:

In [25]:
data.duplicated().sum()

0

Явные дубликаты удалены. Для поиска неявных дубликатов воспользуемся методом value_counts(). Столбцы 'education' и 'income_type' были проверены на этапе обработки пропусков, поэтому их пропускаем и проверяем остальные колонки, содержащие строки: 'family_status', 'gender' и 'purpose'.

Проверим столбец 'family_status':

In [26]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Дубликатов нет, но в категории 'Не женат / не замужем' символы в разных регистрах, поэтому приведем символы к одному стилю.

In [27]:
data['family_status'] = data['family_status'].str.lower()

Проверим столбец 'gender' на неявные дубликаты:

In [28]:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [29]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19947,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Для одного клиента указан пол 'XNA', но так как в столбце 'gender' может быть только женский пол ('F') или мужской ('M'), то значение 'XNA' ошибочное и его можно удалить, так как это всего лишь 0,005% от всей выборки.

Удаляем строчку методом drop() и перезаписываем индексы:

In [30]:
data = data.drop(labels=[19947],axis=0).reset_index(drop=True)

Проверяем, что строчка была удалена:

In [31]:
data['gender'].value_counts()

F    14174
M     7279
Name: gender, dtype: int64

Проверим столбец 'purpose' на неявные дубликаты:

In [32]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце с целями кредита много неявных дубликатов, можно воспользоваться лемматизацией для поиска слов, записанных в разных формах и привести их к словарной форме.

<div style="border: solid green 2px; padding: 20px">

**Вывод**
    
Обнаруженные явные дубликаты были удалены, неявные дубликаты есть в столбце с целями, но так как вручную обрабатывать их неудобно, то применим лемматизацию.</div>

<a name="2.4"><h3>2.4 Лемматизация</h3></a>

Создаём функцию, которая лемматизирует каждое слово из списка, проверяет содержимое lemmas и распределяет по категориям 'свадьба', 'ремонт', 'недвижимость', 'автомобиль', 'образование' или 'другое':

In [33]:
m = Mystem()
def lemmatize(purpose):
    for word in purpose.split():
        lemmas = m.lemmatize(word)
        if 'свадьба' in lemmas:
            return 'свадьба'
        elif 'ремонт' in lemmas:
            return 'ремонт'
        elif 'жилье' in lemmas or 'недвижимость' in lemmas:
            return 'недвижимость'
        elif 'автомобиль' in lemmas:
            return 'автомобиль'
        elif 'образование' in lemmas:
            return 'образование'
    else:
        return 'другое'

Добавляем новый столбец 'lemmatize_purpose' в таблицу и вызываем функцию 'lemmatize' для целей в столбце 'purpose':

In [34]:
data['lemmatize_purpose'] = data['purpose'].apply(lemmatize)

Проверяем столбец 'purpose':

In [35]:
data['lemmatize_purpose'].value_counts()

недвижимость    10203
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: lemmatize_purpose, dtype: int64

Заменяем значения столбца с целями получения кредита на леммы:

In [36]:
data['purpose'] = data['lemmatize_purpose']

Выведем таблицу и проверим, что значения в столбце 'purpose' были заменены:

In [37]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,недвижимость
1,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,свадьба
2,0,2369,33,высшее,0,гражданский брак,1,M,сотрудник,0,90410,недвижимость,недвижимость
3,0,272,21,высшее,0,гражданский брак,1,M,сотрудник,0,128265,свадьба,свадьба
4,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,недвижимость,недвижимость
5,1,717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863,недвижимость,недвижимость
6,0,597,25,высшее,0,не женат / не замужем,4,M,сотрудник,1,192247,образование,образование
7,1,6953,50,высшее,0,гражданский брак,1,F,сотрудник,0,203199,свадьба,свадьба
8,0,3480,27,высшее,0,гражданский брак,1,F,сотрудник,0,92238,автомобиль,автомобиль
9,0,335,36,высшее,0,женат / замужем,0,M,сотрудник,0,414404,недвижимость,недвижимость


Колонка с целями обновлена, поэтому столбец 'lemmatize_purpose' больше не нужен и его можно удалить:

In [38]:
data.drop('lemmatize_purpose', axis=1, inplace=True)

<div style="border: solid green 2px; padding: 20px">
    
**Вывод**

В результате обработки столбца 'purpose' цели заменены на категории: 'недвижимость', 'автомобиль', 'образование', 'свадьба' и 'ремонт'.</div>

<a name="2.5"><h3>2.5 Категоризация данных</h3></a>

Для целей исследования объединим данные в категории по количеству детей, возрасту и уровню дохода.

#### 2.5.1 Категории клиентов по количеству детей

Проверяем столбец с количеством детей клиентов:

In [39]:
data['children'].value_counts()

 0     14090
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке с количеством детей встречаются значения -1 и 20. Минус один ребёнок быть не может, скорее всего это ошибка ввода данных, поэтому заменим значение -1 на 1 ребенка.
Нельзя утверждать, что у семьи не может быть 20 детей, так как возможно усыновление, но все же такие семьи встречаются очень редко, а в таблице таких 76 записей. Вероятнее всего ноль был поставлен ошибочно, поэтому заменим 20 на 2 детей.

In [40]:
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

Проверяем, что некорректных значений больше нет:

In [41]:
data['children'].value_counts()

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Создадим следующие категории:
* 0 детей - 'бездетные'
* до 2 детей - '1-2 ребенка'
* больше 3 - 'многодетные'

Функция для распределения клиентов по количеству детей:

In [42]:
def children_group(number_children):
    if number_children == 0:
        return 'бездетные'
    elif number_children == 1 or number_children == 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

Создаем новый столбец и применяем к колонке 'children' функцию 'children_group':

In [43]:
data['children_group'] = data['children'].apply(children_group)

Проверяем, что новый столбец был добавлен:

In [44]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1-2 ребенка
1,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,1-2 ребенка
2,0,2369,33,высшее,0,гражданский брак,1,M,сотрудник,0,90410,недвижимость,бездетные
3,0,272,21,высшее,0,гражданский брак,1,M,сотрудник,0,128265,свадьба,бездетные
4,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,недвижимость,бездетные


Выведем статистику с группами по количеству детей:

In [45]:
data['children_group'].value_counts()

бездетные      14090
1-2 ребенка     6983
многодетные      380
Name: children_group, dtype: int64

Большинство клиентов бездетные, многодетных всего 1,8%.

#### 2.5.2 Категории клиентов по возрасту

Выберем и отсортируем уникальные значения возраста клиентов для распределения по категориям:

In [46]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

В столбце есть значения с возрастом 0, но так как такого быть не может, заменим их на средний возраст:

In [47]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()

Проверяем значения:

In [48]:
data['dob_years'].sort_values().unique()

array([19.        , 20.        , 21.        , 22.        , 23.        ,
       24.        , 25.        , 26.        , 27.        , 28.        ,
       29.        , 30.        , 31.        , 32.        , 33.        ,
       34.        , 35.        , 36.        , 37.        , 38.        ,
       39.        , 40.        , 41.        , 42.        , 43.        ,
       43.27212977, 44.        , 45.        , 46.        , 47.        ,
       48.        , 49.        , 50.        , 51.        , 52.        ,
       53.        , 54.        , 55.        , 56.        , 57.        ,
       58.        , 59.        , 60.        , 61.        , 62.        ,
       63.        , 64.        , 65.        , 66.        , 67.        ,
       68.        , 69.        , 70.        , 71.        , 72.        ,
       73.        , 74.        , 75.        ])

После замены тип данных стал 'float', поменяем тип в столбце 'dob_years' на целочисленный:

In [49]:
data['dob_years'] = data['dob_years'].astype('int')

Проверяем, что тип данных в столбце 'dob_years' изменился на 'int':

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int64 
 1   days_employed     21453 non-null  int64 
 2   dob_years         21453 non-null  int64 
 3   education         21453 non-null  object
 4   education_id      21453 non-null  int64 
 5   family_status     21453 non-null  object
 6   family_status_id  21453 non-null  int64 
 7   gender            21453 non-null  object
 8   income_type       21453 non-null  object
 9   debt              21453 non-null  int64 
 10  total_income      21453 non-null  int64 
 11  purpose           21453 non-null  object
 12  children_group    21453 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


Выделим категории по возрастам:
* от 19 до 35 лет  - молодой возраст
* от 36 до 59 - средний возраст
* после 60 - пожилой возраст

Функция для распределения клиентов по возрастам:

In [51]:
def years_group(age):
    if age <= 35:
        return 'молодой возраст'
    elif 36 <= age <= 59:
        return 'средний возраст'
    elif age >= 60:
        return 'пожилой возраст'

Создаем новый столбец и применяем к колонке 'dob_years' функцию 'years_group':

In [52]:
data['dob_years_group'] = data['dob_years'].apply(years_group)

Проверяем, что колонка с категориями по возрастам добавлена:

In [53]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,dob_years_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1-2 ребенка,средний возраст
1,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,1-2 ребенка,молодой возраст
2,0,2369,33,высшее,0,гражданский брак,1,M,сотрудник,0,90410,недвижимость,бездетные,молодой возраст
3,0,272,21,высшее,0,гражданский брак,1,M,сотрудник,0,128265,свадьба,бездетные,молодой возраст
4,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,недвижимость,бездетные,молодой возраст


Выведем статистику с группами по возрасту:

In [54]:
data['dob_years_group'].value_counts()

средний возраст    12371
молодой возраст     6582
пожилой возраст     2500
Name: dob_years_group, dtype: int64

Большинство клиентов среднего возраста

#### 2.5.3 Категории клиентов по уровню дохода

Просмотрим основные статистические данные в столбце 'total_income' методом describe():

In [55]:
data['total_income'].describe().apply("{0:.0f}".format)

count      21453
mean      165451
std        98291
min        20667
25%       107515
50%       143707
75%       198301
max      2265604
Name: total_income, dtype: object

Выделим категории по доходам клиентов по процентилям:
* от минимального значения до 25%  - низкий доход
* от 25% до медианы  - средний доход 
* от медианы до 75% - выше среднего
* после 75%  - высокий доход

Распределим клиентов по доходам:

In [56]:
data['total_income_group'] = pd.qcut(data['total_income'], 4, ['низкий доход','средний доход','выше среднего', 'высокий доход'])

Проверяем, что колонка с категориями по доходам добавлена:

In [57]:
data[['total_income','total_income_group']].head(10)

,total_income,total_income_group
0,253875,высокий доход
1,95856,низкий доход
2,90410,низкий доход
3,128265,средний доход
4,308848,высокий доход
5,187863,выше среднего
6,192247,выше среднего
7,203199,высокий доход
8,92238,низкий доход
9,414404,высокий доход


#### 2.5.4 Создание словарей

Создадим словари для уровня образования и семейного статуса, где названию категории будет соответствовать номер.

Словарь 'dict_education':

In [58]:
dict_education = data[['education','education_id']]

Удаляем из словаря дубликаты и перезаписываем индексы:

In [59]:
dict_education = dict_education.drop_duplicates().reset_index(drop=True)

Выводим словарь 'dict_education':

In [60]:
dict_education.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Словарь 'dict_family_status':

In [61]:
dict_family_status = data[['family_status','family_status_id']]

Удаляем из словаря дубликаты и перезаписываем индексы:

In [62]:
dict_family_status = dict_family_status.drop_duplicates().reset_index(drop=True)

Выводим словарь 'dict_family_status':

In [63]:
dict_family_status.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,не женат / не замужем,4
3,вдовец / вдова,2
4,в разводе,3


<div style="border: solid green 2px; padding: 20px">
    
**Вывод**  

В результате проведенной категоризации данных, в DataFrame было добавлено три новых столбца с разделением клиентов на категории, также было создано два словаря для колонок с образованием и семейным статусом.</div>

<a name="3"><h2>3 Ответы на вопросы</h2></a>

<a name="3.1"><h3>3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?</h3></a>

Создаем функцию, которая рассчитывает процент клиентов, имеющих задолженность перед банком, переименовывает колонки и выводит отсортированную таблицу по столбцу 'percentage_debtors':

In [64]:
def func_pivot_table(table):
    table['all_clients'] = table[0] + table[1]
    table['percentage_debtors'] = round(table[1] * 100 / table['all_clients'], 2)
    table = table.set_axis(['no_debt', 'number_debtorse', 'all_clients', 'percentage_debtors'], axis='columns', inplace=False)
    return table.sort_values(by='percentage_debtors', ascending=False)

Выводим сводную таблицу, сгруппированную по количеству детей:

In [65]:
pivot_children = data.pivot_table(index='children_group', columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_children)

,no_debt,number_debtorse,all_clients,percentage_debtors
children_group,,,,
1-2 ребенка,6336,647,6983,9.27
многодетные,349,31,380,8.16
бездетные,13027,1063,14090,7.54


<div style="border: solid green 2px; padding: 20px">
    
**Вывод**

Бездетные клиенты лучше возвращают кредит в срок. Клиенты, имеющие 1 или 2 детей, являются наименее исполнительными.
</div>

<a name="3.2"><h3>3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?</h3></a>

Создаём сводную таблицу, сгруппированную по семейному статусу:

In [66]:
pivot_family_status = data.pivot_table(index='family_status', columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_family_status)

,no_debt,number_debtorse,all_clients,percentage_debtors
family_status,,,,
не женат / не замужем,2536,274,2810,9.75
гражданский брак,3762,388,4150,9.35
женат / замужем,11408,931,12339,7.55
в разводе,1110,85,1195,7.11
вдовец / вдова,896,63,959,6.57


Клиенты в категории 'вдовец / вдова' лучше возвращают долг, самый высокий показатель по задолженности у клиентов 'не женат / не замужем' и 'гражданский брак'.

Рассмотрим также сводную таблицу с группировкой по семейному статусу и количеству детей:

In [67]:
pivot_family_children = data.pivot_table(index=['family_status','children_group'], columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_family_children)

,,no_debt,number_debtorse,all_clients,percentage_debtors
family_status,children_group,,,,
не женат / не замужем,многодетные,8.0,2.0,10.0,20.00
гражданский брак,многодетные,58.0,8.0,66.0,12.12
не женат / не замужем,1-2 ребенка,476.0,62.0,538.0,11.52
гражданский брак,1-2 ребенка,1204.0,151.0,1355.0,11.14
вдовец / вдова,1-2 ребенка,95.0,10.0,105.0,9.52
не женат / не замужем,бездетные,2052.0,210.0,2262.0,9.28
женат / замужем,1-2 ребенка,4191.0,395.0,4586.0,8.61
гражданский брак,бездетные,2500.0,229.0,2729.0,8.39
в разводе,многодетные,11.0,1.0,12.0,8.33


Самый большой процент задолженности у многодетных клиентов в категории 'не женат/ не замужем', но при этом очень маленькая выборка, всего 10 человек, поэтому такие данные ненадёжные. Но если посмотреть на верхние строки, то всё же можно сказать, что клиенты с детьми и в категории 'не женат / не замужем' или 'гражданский брак' хуже возвращают долги по кредитам.  Самый маленький процент по задолженности у бездетных клиентов в категории 'женат / замужем', 'вдовец / вдова' или 'в разводе'.

Посчитаем общий процент задолженности у клиентов с детьми в категории 'не женат/ не замужем'.

Для этого сортируем таблицу по категориям 'не женат/ не замужем' и не 'бездетные':

In [68]:
not_married = data[(data['family_status'] == 'не женат / не замужем') & (data['children_group'] != 'бездетные')]

Выводим сводную таблицу:

In [69]:
pivot_family_children = not_married.pivot_table(index=['family_status'], columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_family_children)

,no_debt,number_debtorse,all_clients,percentage_debtors
family_status,,,,
не женат / не замужем,484,64,548,11.68


Посчитаем общий процент задолженности у клиентов с детьми в категории 'гражданский брак'.

Cортируем таблицу по категориям 'гражданский брак' и не 'бездетные':

In [70]:
civil_marriage = data[(data['family_status'] == 'гражданский брак') & (data['children_group'] != 'бездетные')]

Выводим сводную таблицу:

In [71]:
pivot_family_children = civil_marriage.pivot_table(index=['family_status'], columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_family_children)

,no_debt,number_debtorse,all_clients,percentage_debtors
family_status,,,,
гражданский брак,1262,159,1421,11.19


У клиентов с семейным статусом "не женат / не замужем" и "гражданский брак" процент задолженности составляет 11.68% и 11,19% соответственно.

<div style="border: solid green 2px; padding: 20px">
    
**Вывод**

Семейное положение влияет на возврат кредита. Наименьший процент долга у клиентов в категории 'вдовец / вдова'.</div>

<a name="3.3"><h3>3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</h3></a>

Создаём сводную таблицу, сгруппированную по уровню дохода:

In [72]:
pivot_total_income = data.pivot_table(index='total_income_group', columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_total_income)

,no_debt,number_debtorse,all_clients,percentage_debtors
total_income_group,,,,
средний доход,4878,485,5363,9.04
выше среднего,4907,456,5363,8.50
низкий доход,4937,427,5364,7.96
высокий доход,4990,373,5363,6.96


<div style="border: solid green 2px; padding: 20px">
    
**Вывод**

Клиенты с высоким доходом лучше возвращают долги, менее надежная категория клиентов со средним доходом, но при этом показатели клиентов из категорий с доходом выше среднего и низким доходом не сильно отличаются от клиентов с категорией 'средний доход'.</div>

<a name="3.4"><h3>3.4 Как разные цели кредита влияют на его возврат в срок?</h3></a>

Создаём сводную таблицу, сгруппированную по целям кредита:

In [73]:
pivot_purpose = data.pivot_table(index='purpose', columns='debt', values='education_id', aggfunc='count')
func_pivot_table(pivot_purpose)

,no_debt,number_debtorse,all_clients,percentage_debtors
purpose,,,,
автомобиль,3903,403,4306,9.36
образование,3643,370,4013,9.22
свадьба,2138,186,2324,8.00
недвижимость,9456,747,10203,7.32
ремонт,572,35,607,5.77


<div style="border: solid green 2px; padding: 20px">
    
**Вывод**

Самый низкий показатель по задолженности в категории 'ремонт', кредиты на цели 'автомобиль' и 'образование' погашаются хуже, чем в случае с другими целями.</div>

<a name="4"><h2>4 Общие выводы</h2></a>

Проанализировав статистику платежеспособности клиентов, можно сказать, что семейное положение, количество детей, доход клиентов и цели кредита влияют на факт погашения кредитной задолженности.
    
Результаты исследования показали:
    
   - Самые надежные клиенты это люди, не имеющие детей, в категории 'вдовец / вдова' (6.26%), 'женат / замужем' (6.9%) или 'в разводе' (7.01%), а самые безответственные это клиенты с детьми и в категории 'не женат / не замужем' (11.67%) или 'гражданский брак' (11.19%).
   - По уровню дохода определили, что лучше всего клиенты возвращают долг в категории с высоким доходом (6.96%), клиенты с наибольшей задолженностью имеют средний доход (9.04%) и доход выше среднего (8.5%).
   - По целям самый низкий процент задолженности у клиентов, берущих кредит на оплату ремонта (5.77%), а самый высокий у тех, кто обращается в банк для покупки автомобиля (9.36%) или оплаты образования (9.22%).